In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import json
from datetime import datetime


from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass


In [46]:
options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")


driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
# time.sleep(1)
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
# time.sleep(1)
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

In [47]:
link = driver.find_element(By.CSS_SELECTOR, "a img[alt='外国株式・海外ETF']")
driver.execute_script("arguments[0].click();", link)
time.sleep(0.3)

In [48]:
# JavaScriptを使用してクリックする
link = driver.find_element(By.CSS_SELECTOR, "a._mypage_btn__8wnX_")
driver.execute_script("arguments[0].click();", link)
time.sleep(0.3)

In [49]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)
time.sleep(0.5)

In [72]:
input_element = driver.find_element(By.CLASS_NAME, "input-sm")
input_element.send_keys("NVDA")
time.sleep(0.3)

In [73]:
search_button = driver.find_element(By.CSS_SELECTOR, "button[data-ga-button='brandsearch']")

search_button.click()
time.sleep(0.3)

In [74]:
# 'ニュース'タブに対応するボタンを見つける
news_tab = driver.find_element(By.CSS_SELECTOR, "button[data-ga-tab='news']")

# ボタンをクリックする
news_tab.click()
time.sleep(0.3)

In [75]:
iframe = driver.page_source

soup = BeautifulSoup(iframe, "html.parser")
url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"

# Search for iframes or similar tags that might contain the specified URL pattern
iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

# Display the found URLs from iframes
iframe_urls

['https://graph.sbisec.co.jp/sbinews/pc?symbol=NVDA&token=2B18F37755EE701E96D35A6ECDAD7E72B59CFE5C62EC6C4EDF7661ED15675FA55444B2645F03F755A65AC2B8E40A1F2F4BABB7A2255C9B4E6D9C2D5E9E35CEBDC4918250C7D70338863EA4696F9031A0']

In [76]:
print(iframe_urls[0])
driver.execute_script(f"window.open('{iframe_urls[0]}', '_blank');")

https://graph.sbisec.co.jp/sbinews/pc?symbol=NVDA&token=2B18F37755EE701E96D35A6ECDAD7E72B59CFE5C62EC6C4EDF7661ED15675FA55444B2645F03F755A65AC2B8E40A1F2F4BABB7A2255C9B4E6D9C2D5E9E35CEBDC4918250C7D70338863EA4696F9031A0


In [77]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][1]
driver.switch_to.window(new_window)
time.sleep(0.5)

In [78]:
prev_to_next_button = driver.find_element(By.CSS_SELECTOR, "li.paginationjs-page.paginationjs-last.J-paginationjs-page a")
prev_to_next_button.click()


In [38]:
# print(driver.page_source)

In [80]:


# 今日の日付を取得してファイル名に使用
today_date = datetime.now().strftime("%Y-%m-%d")
filename = f"./scrape_data/{today_date}_test_data.json"

# WebDriverWaitの設定
wait = WebDriverWait(driver, 10)

def collect_news_data():
    news_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "data-row")))
    news_data = []

    for item in reversed(news_items):
        # ニュース項目をクリック
        wait.until(EC.element_to_be_clickable(item)).click()

        # コンテンツが表示されるのを待つ
        content = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".data-row[is-open='true'] .content")))
        # time.sleep(0.02)  # 展開待ち

        headline = item.find_element(By.CLASS_NAME, "headline").text.strip()
        date_time = item.find_element(By.CLASS_NAME, "date").text.strip()
        content_text = content.text.strip() if content.text.strip() else "No content available"

        news_data.append({"headline": headline, "date": date_time, "content": content_text})

    return news_data

# 初回のニュースデータ収集
all_news_data = collect_news_data()
print(all_news_data)

# 「Previous page」の処理
while True:
    try:
        prev_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#pagination-container1 .paginationjs-prev:not(.disabled)")))
        if "disabled" in prev_button.get_attribute("class"):
            print("Reached the first page, no more previous pages.")
            break
        prev_button.click()
        wait.until(EC.staleness_of(news_items[0]))
        new_data = collect_news_data()
        all_news_data.extend(new_data)
        print(new_data)
    except:
        break


# JSONファイルに保存
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(all_news_data, f, ensure_ascii=False, indent=4)


NameError: name 'news_items' is not defined

In [ ]:

driver.close()
driver.switch_to.window(original_window)

driver.find_element(By.ID, "logo-icon").click()

In [ ]:
def load_symbols(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    symbols = []
    for entry in data:
        symbols.append(entry["symbol"])
    return symbols

In [81]:

def news(n_stock):
    input_element = driver.find_element(By.CLASS_NAME, "input-sm")
    input_element.send_keys(n_stock)
    time.sleep(0.3)

    search_button = driver.find_element(By.CSS_SELECTOR, "button[data-ga-button='brandsearch']")

    search_button.click()
    time.sleep(0.3)

    # 'ニュース'タブに対応するボタンを見つける
    news_tab = driver.find_element(By.CSS_SELECTOR, "button[data-ga-tab='news']")

    # ボタンをクリックする
    news_tab.click()
    time.sleep(0.3)

    iframe = driver.page_source

    soup = BeautifulSoup(iframe, "html.parser")
    url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"

    # Search for iframes or similar tags that might contain the specified URL pattern
    iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

    driver.execute_script(f"window.open('{iframe_urls[0]}', '_blank');")

    # 現在のウィンドウハンドルを取得
    original_window = driver.current_window_handle

    # 新しいタブまたはウィンドウに切り替え
    new_window = [window for window in driver.window_handles if window != original_window][1]
    driver.switch_to.window(new_window)
    time.sleep(0.5)

    prev_to_next_button = driver.find_element(By.CSS_SELECTOR, "li.paginationjs-page.paginationjs-last.J-paginationjs-page a")
    prev_to_next_button.click()

    # 今日の日付を取得してファイル名に使用
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f"./scrape_data/{n_stock}_{today_date}_news_data.json"

    # WebDriverWaitの設定
    wait = WebDriverWait(driver, 10)

    news_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "data-row")))
    news_data = []

    for item in reversed(news_items):
        # ニュース項目をクリック
        wait.until(EC.element_to_be_clickable(item)).click()

        # コンテンツが表示されるのを待つ
        content = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".data-row[is-open='true'] .content")))
        # time.sleep(0.02)  # 展開待ち

        headline = item.find_element(By.CLASS_NAME, "headline").text.strip()
        date_time = item.find_element(By.CLASS_NAME, "date").text.strip()
        content_text = content.text.strip() if content.text.strip() else "No content available"

        news_data.append({"headline": headline, "date": date_time, "content": content_text})


    # 初回のニュースデータ収集
    all_news_data = news_data
    print(all_news_data)

    # 「Previous page」の処理
    while True:
        try:
            prev_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#pagination-container1 .paginationjs-prev:not(.disabled)")))
            if "disabled" in prev_button.get_attribute("class"):
                print("Reached the first page, no more previous pages.")
                break
            prev_button.click()
            wait.until(EC.staleness_of(news_items[0]))
            new_data = collect_news_data()
            all_news_data.extend(new_data)
            print(new_data)
        except:
            break


    # JSONファイルに保存
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(all_news_data, f, ensure_ascii=False, indent=4)

    driver.close()
    driver.switch_to.window(original_window)

    driver.find_element(By.ID, "logo-icon").click()



In [83]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)
time.sleep(0.5)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF76AF01502+60802]
	(No symbol) [0x00007FF76AE7AC02]
	(No symbol) [0x00007FF76AD37CE4]
	(No symbol) [0x00007FF76AD0DFDF]
	(No symbol) [0x00007FF76ADB1E57]
	(No symbol) [0x00007FF76ADB8DDB]
	(No symbol) [0x00007FF76ADAAB40]
	(No symbol) [0x00007FF76AD78FEC]
	(No symbol) [0x00007FF76AD79C21]
	GetHandleVerifier [0x00007FF76B20411D+3217821]
	GetHandleVerifier [0x00007FF76B2460B7+3488055]
	GetHandleVerifier [0x00007FF76B23F03F+3459263]
	GetHandleVerifier [0x00007FF76AFBB846+823494]
	(No symbol) [0x00007FF76AE85F9F]
	(No symbol) [0x00007FF76AE80EC4]
	(No symbol) [0x00007FF76AE81052]
	(No symbol) [0x00007FF76AE718A4]
	BaseThreadInitThunk [0x00007FFE7FE5257D+29]
	RtlUserThreadStart [0x00007FFE807CAA48+40]


In [82]:
news("NVDA")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF76AF01502+60802]
	(No symbol) [0x00007FF76AE7AC02]
	(No symbol) [0x00007FF76AD37CE4]
	(No symbol) [0x00007FF76AD0DFDF]
	(No symbol) [0x00007FF76ADB1E57]
	(No symbol) [0x00007FF76ADC98D1]
	(No symbol) [0x00007FF76ADAA923]
	(No symbol) [0x00007FF76AD78FEC]
	(No symbol) [0x00007FF76AD79C21]
	GetHandleVerifier [0x00007FF76B20411D+3217821]
	GetHandleVerifier [0x00007FF76B2460B7+3488055]
	GetHandleVerifier [0x00007FF76B23F03F+3459263]
	GetHandleVerifier [0x00007FF76AFBB846+823494]
	(No symbol) [0x00007FF76AE85F9F]
	(No symbol) [0x00007FF76AE80EC4]
	(No symbol) [0x00007FF76AE81052]
	(No symbol) [0x00007FF76AE718A4]
	BaseThreadInitThunk [0x00007FFE7FE5257D+29]
	RtlUserThreadStart [0x00007FFE807CAA48+40]
